In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from dbapi import DBManager

In [39]:
from tqdm.notebook import tqdm

In [3]:
DB_FILEPATH = "D:\\Masatoshi\\Work\\db\\keiba.db"

In [4]:
dbm = DBManager(DB_FILEPATH)

In [76]:
results = dbm.get_united_results()
results

,race_id,horse_no,frame_no,arriving_order,horse_id,sex_age,impost,jockey_id,goal_time,corner_pass,last_three_furlong,win_odds,popular_order,horse_weight,trainer_id,owner_name,date,place_id,race_no,distance,race_type,turn,ground,weather,father,mother,fathers_father,fathers_mother,mothers_father,mothers_mother
0,202101010101,9,5,1,2018105875,牝3,52.0,01181,1:08.7,4-4,34.6,13.5,7.0,474(-22),01185,松岡隆雄,20210612,1,1,1200,芝,右,良,小雨,ダイワメジャー,カシュカシュ,サンデーサイレンス,スカーレットブーケ,アフリート,スリーソウツ
1,202101010101,6,3,2,2018105258,牡3,56.0,01140,1:08.8,1-1,35.0,3.9,2.0,470(-2),00438,キャロットファーム,20210612,1,1,1200,芝,右,良,小雨,エピファネイア,ペルレンケッテ,シンボリクリスエス,シーザリオ,ディープインパクト,プンティラ
2,202101010101,14,7,3,2018101134,牝3,54.0,01096,1:08.9,11-10,34.0,124.7,14.0,394(+2),01123,永田和彦,20210612,1,1,1200,芝,右,良,小雨,ミッキーアイル,ザッハトルテ,ディープインパクト,スターアイル,クロフネ,パフェ
3,202101010101,10,5,4,2018102791,牝3,54.0,01157,1:09.0,2-2,35.2,7.6,5.0,468(+2),01088,了徳寺健二ホールディングス,20210612,1,1,1200,芝,右,良,小雨,Kingman,ドゥケシーナ,Invincible Spirit,Zenda,Henrythenavigator,Dreamlike
4,202101010101,5,3,5,2018101539,牝3,54.0,01015,1:09.0,7-7,34.5,7.5,4.0,456(+8),00417,北所直人,20210612,1,1,1200,芝,右,良,小雨,カレンブラックヒル,ヒシシャトル,ダイワメジャー,チャールストンハーバー,ボストンハーバー,ヒシレイホウ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336239,201510021212,14,7,12,2012102559,牡3,54.0,01141,2:43.7,14-16-14-15,36.6,110.0,13.0,486(+6),01021,下村直,20150906,10,12,2600,芝,右,稍,小雨,バトルプラン,マーブルダンサー,エンパイアメーカー,Flanders,ダンスインザダーク,レアシングチェリー
336240,201510021212,3,2,13,2011103862,牡4,56.0,01154,2:43.9,1-1-1-1,38.1,14.9,6.0,494(+4),01053,東京ホースレーシング,20150906,10,12,2600,芝,右,稍,小雨,ゼンノロブロイ,サセッティ,サンデーサイレンス,ローミンレイチェル,Selkirk,My Potters
336241,201510021212,8,4,14,2012101695,牡3,51.0,01155,2:43.9,13-12-6-10,37.7,174.9,15.0,452(-8),00224,森中蕃,20150906,10,12,2600,芝,右,稍,小雨,グラスワンダー,セレスフェアリー,Silver Hawk,Ameriflora,アドマイヤベガ,ユーキャンフライ
336242,201510021212,12,6,15,2012102903,牡3,54.0,01130,2:44.2,2-2-2-3,38.3,138.6,14.0,426(-6),01041,中西宏彰,20150906,10,12,2600,芝,右,稍,小雨,ヴィクトリー,ケンブリッジマイア,ブライアンズタイム,グレースアドマイヤ,アグネスタキオン,ノーザンマイア


In [77]:
from data_processor import preprocesing
results_p = preprocesing(results)
results_p.head()

  0%|          | 0/23797 [00:00<?, ?it/s]

,race_id,horse_no,frame_no,arriving_order,horse_id,impost,jockey_id,goal_time,corner_pass,last_three_furlong,win_odds,popular_order,trainer_id,owner_name,date,place_id,race_no,distance,race_type,turn,ground,weather,father,mother,fathers_father,fathers_mother,mothers_father,mothers_mother,rank,sex,age,weigth,weight_change,horse_num,time_diff
0,202101010101,9,5,1,2018105875,52.0,01181,1900-01-01 00:01:08.700,4-4,34.6,13.5,7.0,01185,松岡隆雄,2021-06-12,1,1,1200,芝,右,良,小雨,ダイワメジャー,カシュカシュ,サンデーサイレンス,スカーレットブーケ,アフリート,スリーソウツ,0,牝,3,474,-22,16,0.0
1,202101010101,6,3,2,2018105258,56.0,01140,1900-01-01 00:01:08.800,1-1,35.0,3.9,2.0,00438,キャロットファーム,2021-06-12,1,1,1200,芝,右,良,小雨,エピファネイア,ペルレンケッテ,シンボリクリスエス,シーザリオ,ディープインパクト,プンティラ,0,牡,3,470,-2,16,0.1
2,202101010101,14,7,3,2018101134,54.0,01096,1900-01-01 00:01:08.900,11-10,34.0,124.7,14.0,01123,永田和彦,2021-06-12,1,1,1200,芝,右,良,小雨,ミッキーアイル,ザッハトルテ,ディープインパクト,スターアイル,クロフネ,パフェ,0,牝,3,394,2,16,0.2
3,202101010101,10,5,4,2018102791,54.0,01157,1900-01-01 00:01:09.000,2-2,35.2,7.6,5.0,01088,了徳寺健二ホールディングス,2021-06-12,1,1,1200,芝,右,良,小雨,Kingman,ドゥケシーナ,Invincible Spirit,Zenda,Henrythenavigator,Dreamlike,1,牝,3,468,2,16,0.3
4,202101010101,5,3,5,2018101539,54.0,01015,1900-01-01 00:01:09.000,7-7,34.5,7.5,4.0,00417,北所直人,2021-06-12,1,1,1200,芝,右,良,小雨,カレンブラックヒル,ヒシシャトル,ダイワメジャー,チャールストンハーバー,ボストンハーバー,ヒシレイホウ,1,牝,3,456,8,16,0.3


In [78]:
import datetime
target_df = results_p[results_p['race_type'] != '障害'].copy()
target_df = target_df[target_df['date'] >= pd.to_datetime('2020/1/1')]
target_df

,race_id,horse_no,frame_no,arriving_order,horse_id,impost,jockey_id,goal_time,corner_pass,last_three_furlong,win_odds,popular_order,trainer_id,owner_name,date,place_id,race_no,distance,race_type,turn,ground,weather,father,mother,fathers_father,fathers_mother,mothers_father,mothers_mother,rank,sex,age,weigth,weight_change,horse_num,time_diff
0,202101010101,9,5,1,2018105875,52.0,01181,1900-01-01 00:01:08.700,4-4,34.6,13.5,7.0,01185,松岡隆雄,2021-06-12,1,1,1200,芝,右,良,小雨,ダイワメジャー,カシュカシュ,サンデーサイレンス,スカーレットブーケ,アフリート,スリーソウツ,0,牝,3,474,-22,16,0.0
1,202101010101,6,3,2,2018105258,56.0,01140,1900-01-01 00:01:08.800,1-1,35.0,3.9,2.0,00438,キャロットファーム,2021-06-12,1,1,1200,芝,右,良,小雨,エピファネイア,ペルレンケッテ,シンボリクリスエス,シーザリオ,ディープインパクト,プンティラ,0,牡,3,470,-2,16,0.1
2,202101010101,14,7,3,2018101134,54.0,01096,1900-01-01 00:01:08.900,11-10,34.0,124.7,14.0,01123,永田和彦,2021-06-12,1,1,1200,芝,右,良,小雨,ミッキーアイル,ザッハトルテ,ディープインパクト,スターアイル,クロフネ,パフェ,0,牝,3,394,2,16,0.2
3,202101010101,10,5,4,2018102791,54.0,01157,1900-01-01 00:01:09.000,2-2,35.2,7.6,5.0,01088,了徳寺健二ホールディングス,2021-06-12,1,1,1200,芝,右,良,小雨,Kingman,ドゥケシーナ,Invincible Spirit,Zenda,Henrythenavigator,Dreamlike,1,牝,3,468,2,16,0.3
4,202101010101,5,3,5,2018101539,54.0,01015,1900-01-01 00:01:09.000,7-7,34.5,7.5,4.0,00417,北所直人,2021-06-12,1,1,1200,芝,右,良,小雨,カレンブラックヒル,ヒシシャトル,ダイワメジャー,チャールストンハーバー,ボストンハーバー,ヒシレイホウ,1,牝,3,456,8,16,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90680,202010020812,1,1,11,2016105042,57.0,00666,1900-01-01 00:01:45.500,14-12-5-6,37.7,6.5,4.0,01071,キーファーズ,2020-09-06,10,12,1700,ダート,右,重,雨,ディープインパクト,シャンパンドーロ,サンデーサイレンス,ウインドインハーヘア,Medaglia d'Oro,Champagne Glow,1,牡,4,476,-2,15,2.3
90681,202010020812,14,8,12,2016110126,57.0,01115,1900-01-01 00:01:45.600,3-3-2-3,38.0,19.8,8.0,01022,吉田和美,2020-09-06,10,12,1700,ダート,右,重,雨,Itsmyluckyday,Briecat,Lawyer Ron,Viva La Slew,Adcat,Silk Briefcase,1,牡,4,522,-8,15,2.4
90682,202010020812,15,8,13,2015102639,57.0,01124,1900-01-01 00:01:45.900,13-12-13-13,37.3,75.0,14.0,00429,コウトミックレーシング,2020-09-06,10,12,1700,ダート,右,重,雨,パイロ,オーゴンサンデー,Pulpit,Wild Vision,マーベラスサンデー,ピサノダイヤ,1,牡,5,510,0,15,2.7
90683,202010020812,3,2,14,2016110067,55.0,01037,1900-01-01 00:01:46.700,15-15-15-14,37.8,46.1,10.0,01164,ノースヒルズ,2020-09-06,10,12,1700,ダート,右,重,雨,Temple City,Malibu Pier,Dynaformer,Curriculum,Malibu Moon,Blue Moon,1,牝,4,488,-12,15,3.5


In [79]:
target_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87184 entries, 0 to 90684
Data columns (total 35 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   race_id             87184 non-null  int64         
 1   horse_no            87184 non-null  int64         
 2   frame_no            87184 non-null  int64         
 3   arriving_order      87184 non-null  int32         
 4   horse_id            87184 non-null  object        
 5   impost              87184 non-null  float64       
 6   jockey_id           87184 non-null  object        
 7   goal_time           87184 non-null  datetime64[ns]
 8   corner_pass         87184 non-null  object        
 9   last_three_furlong  87184 non-null  float64       
 10  win_odds            87184 non-null  float64       
 11  popular_order       87184 non-null  float64       
 12  trainer_id          87184 non-null  object        
 13  owner_name          87184 non-null  object    

In [103]:
date_list = target_df['date'].unique()
merged_df = pd.DataFrame()
n_samples = 2
for date in date_list:
    print(date)
    date_df = target_df[target_df['date']==date]
    horse_id_list = date_df['horse_id']
    old_results = results_p.query('horse_id in @horse_id_list')
    old_results = old_results.set_index('horse_id')
    filtered_df = old_results[old_results['date'] < date].sort_values('date', ascending=False).groupby(level=0).head(n_samples)
    target_cols = ['place_id', 'weather', 'race_no', 'horse_num', 'horse_no', 'arriving_order', 'race_type', 'distance', 'ground', 'goal_time', 'time_diff', 'last_three_furlong']
    p_grouped_data = filtered_df.groupby(level=0)[target_cols]
    p_data = pd.DataFrame()
    for n in range(n_samples):
        pn_data = p_grouped_data.nth(n).add_prefix('p{}_'.format(n + 1))
        p_data = pd.concat([p_data, pn_data], axis=1)
    merged_df = date_df.merge(p_data, left_on='horse_id', right_index=True, how='left')
    print(merged_df)
    break

2021-06-12T00:00:00.000000000
            race_id  horse_no  ...  p2_time_diff  p2_last_three_furlong
0      202101010101         9  ...           NaN                    NaN
1      202101010101         6  ...           0.3                   34.5
2      202101010101        14  ...           1.3                   35.5
3      202101010101        10  ...           0.2                   35.8
4      202101010101         5  ...           0.5                   38.5
...             ...       ...  ...           ...                    ...
30386  202107040312         9  ...           0.3                   37.5
30387  202107040312         1  ...           0.2                   37.3
30388  202107040312         2  ...           1.3                   34.2
30389  202107040312        11  ...           0.1                   38.7
30390  202107040312        12  ...           0.2                   36.8

[468 rows x 155 columns]


In [104]:
merged_df

,race_id,horse_no,frame_no,arriving_order,horse_id,impost,jockey_id,goal_time,corner_pass,last_three_furlong,win_odds,popular_order,trainer_id,owner_name,date,place_id,race_no,distance,race_type,turn,ground,weather,father,mother,fathers_father,fathers_mother,mothers_father,mothers_mother,rank,sex,age,weigth,weight_change,horse_num,time_diff,p1_place_id,p1_weather,p1_race_no,p1_horse_num,p1_horse_no,...,p1_ground,p1_goal_time,p1_time_diff,p1_last_three_furlong,p2_place_id,p2_weather,p2_race_no,p2_horse_num,p2_horse_no,p2_arriving_order,p2_race_type,p2_distance,p2_ground,p2_goal_time,p2_time_diff,p2_last_three_furlong,p1_place_id,p1_weather,p1_race_no,p1_horse_num,p1_horse_no,p1_arriving_order,p1_race_type,p1_distance,p1_ground,p1_goal_time,p1_time_diff,p1_last_three_furlong,p2_place_id,p2_weather,p2_race_no,p2_horse_num,p2_horse_no,p2_arriving_order,p2_race_type,p2_distance,p2_ground,p2_goal_time,p2_time_diff,p2_last_three_furlong
0,202101010101,9,5,1,2018105875,52.0,01181,1900-01-01 00:01:08.700,4-4,34.6,13.5,7.0,01185,松岡隆雄,2021-06-12,1,1,1200,芝,右,良,小雨,ダイワメジャー,カシュカシュ,サンデーサイレンス,スカーレットブーケ,アフリート,スリーソウツ,0,牝,3,474,-22,16,0.0,NaN,NaN,NaN,NaN,NaN,...,良,1900-01-01 00:01:10.400,0.6,35.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,4.0,晴,5.0,16.0,16.0,6.0,芝,1200.0,良,1900-01-01 00:01:10.400,0.6,35.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1,202101010101,6,3,2,2018105258,56.0,01140,1900-01-01 00:01:08.800,1-1,35.0,3.9,2.0,00438,キャロットファーム,2021-06-12,1,1,1200,芝,右,良,小雨,エピファネイア,ペルレンケッテ,シンボリクリスエス,シーザリオ,ディープインパクト,プンティラ,0,牡,3,470,-2,16,0.1,NaN,NaN,NaN,NaN,NaN,...,良,1900-01-01 00:01:25.600,0.4,37.8,9.0,晴,5.0,16.0,9.0,4.0,芝,1600.0,良,1900-01-01 00:01:34.400,0.3,34.5,9.0,晴,2.0,15.0,15.0,2.0,ダート,1400.0,良,1900-01-01 00:01:25.600,0.4,37.8,9.0,晴,5.0,16.0,9.0,4.0,芝,1600.0,良,1900-01-01 00:01:34.400,0.3,34.5
2,202101010101,14,7,3,2018101134,54.0,01096,1900-01-01 00:01:08.900,11-10,34.0,124.7,14.0,01123,永田和彦,2021-06-12,1,1,1200,芝,右,良,小雨,ミッキーアイル,ザッハトルテ,ディープインパクト,スターアイル,クロフネ,パフェ,0,牝,3,394,2,16,0.2,NaN,NaN,NaN,NaN,NaN,...,不,1900-01-01 00:01:27.200,1.7,39.3,9.0,晴,2.0,12.0,12.0,8.0,芝,1600.0,良,1900-01-01 00:01:35.300,1.3,35.5,7.0,雨,3.0,18.0,17.0,10.0,芝,1400.0,不,1900-01-01 00:01:27.200,1.7,39.3,9.0,晴,2.0,12.0,12.0,8.0,芝,1600.0,良,1900-01-01 00:01:35.300,1.3,35.5
3,202101010101,10,5,4,2018102791,54.0,01157,1900-01-01 00:01:09.000,2-2,35.2,7.6,5.0,01088,了徳寺健二ホールディングス,2021-06-12,1,1,1200,芝,右,良,小雨,Kingman,ドゥケシーナ,Invincible Spirit,Zenda,Henrythenavigator,Dreamlike,1,牝,3,468,2,16,0.3,NaN,NaN,NaN,NaN,NaN,...,良,1900-01-01 00:01:09.200,0.3,35.4,10.0,晴,2.0,18.0,18.0,3.0,芝,1200.0,良,1900-01-01 00:01:08.400,0.2,35.8,10.0,晴,2.0,18.0,9.0,8.0,芝,1200.0,良,1900-01-01 00:01:09.200,0.3,35.4,10.0,晴,2.0,18.0,18.0,3.0,芝,1200.0,良,1900-01-01 00:01:08.400,0.2,35.8
4,202101010101,5,3,5,2018101539,54.0,01015,1900-01-01 00:01:09.000,7-7,34.5,7.5,4.0,00417,北所直人,2021-06-12,1,1,1200,芝,右,良,小雨,カレンブラックヒル,ヒシシャトル,ダイワメジャー,チャールストンハーバー,ボストンハーバー,ヒシレイホウ,1,牝,3,456,8,16,0.3,NaN,NaN,NaN,NaN,NaN,...,良,1900-01-01 00:01:12.700,0.1,37.6,4.0,曇,2.0,15.0,13.0,5.0,ダート,1200.0,良,1900-01-01 00:01:13.200,0.5,38.5,4.0,晴,1.0,15.0,3.0,4.0,ダート,1200.0,良,1900-01-01 00:01:12.700,0.1,37.6,4.0,曇,2.0,15.0,13.0,5.0,ダート,1200.0,良,1900-01-01 00:01:13.200,0.5,38.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30386,202107040312,9,6,9,2017103237,52.0,01186,1900-01-01 00:01:54.600,7-11-11-11,38.1,10.0,5.0,01136,サトミホースカンパニー,2021-06-12,7,12,1800,ダート,左,良,晴,マジェスティックウォリアー,ウエスタンダンサー,A.P. Indy,Dream Supreme,デヒア,ウエスタンローズ,1,牝,4,466,-8,13,2.2,NaN,NaN,NaN,NaN,NaN,...,良,1900-01-01 00:01:53.100,1.0,37.0,9.0,雨,7.0,15.0,4.0,2.0,ダート,1800.0,良,1900-01-01 00:01:52.900,0.3,37.5,4.0,晴,12.0,15.0,8.0,8.0,ダート,1800.0,良,1900-01-01 00:01:53.100,1.0,37.0,9.0,雨,7.0,15.0,4.0,2.0,ダ

In [105]:
from data_processor import Results

In [292]:
r = Results(results)
r.data.head()

,race_id,horse_no,frame_no,arriving_order,horse_id,sex_age,impost,jockey_id,goal_time,corner_pass,last_three_furlong,win_odds,popular_order,horse_weight,trainer_id,owner_name,date,place_id,race_no,distance,race_type,turn,ground,weather,father,mother,fathers_father,fathers_mother,mothers_father,mothers_mother
0,202101010101,9,5,1,2018105875,牝3,52.0,01181,1:08.7,4-4,34.6,13.5,7.0,474(-22),01185,松岡隆雄,20210612,1,1,1200,芝,右,良,小雨,ダイワメジャー,カシュカシュ,サンデーサイレンス,スカーレットブーケ,アフリート,スリーソウツ
1,202101010101,6,3,2,2018105258,牡3,56.0,01140,1:08.8,1-1,35.0,3.9,2.0,470(-2),00438,キャロットファーム,20210612,1,1,1200,芝,右,良,小雨,エピファネイア,ペルレンケッテ,シンボリクリスエス,シーザリオ,ディープインパクト,プンティラ
2,202101010101,14,7,3,2018101134,牝3,54.0,01096,1:08.9,11-10,34.0,124.7,14.0,394(+2),01123,永田和彦,20210612,1,1,1200,芝,右,良,小雨,ミッキーアイル,ザッハトルテ,ディープインパクト,スターアイル,クロフネ,パフェ
3,202101010101,10,5,4,2018102791,牝3,54.0,01157,1:09.0,2-2,35.2,7.6,5.0,468(+2),01088,了徳寺健二ホールディングス,20210612,1,1,1200,芝,右,良,小雨,Kingman,ドゥケシーナ,Invincible Spirit,Zenda,Henrythenavigator,Dreamlike
4,202101010101,5,3,5,2018101539,牝3,54.0,01015,1:09.0,7-7,34.5,7.5,4.0,456(+8),00417,北所直人,20210612,1,1,1200,芝,右,良,小雨,カレンブラックヒル,ヒシシャトル,ダイワメジャー,チャールストンハーバー,ボストンハーバー,ヒシレイホウ


In [293]:
r.preprocesing()
r.data_p.head()

  0%|          | 0/22922 [00:00<?, ?it/s]

,race_id,horse_no,frame_no,arriving_order,horse_id,impost,jockey_id,goal_time,last_three_furlong,trainer_id,date,place_id,race_no,distance,race_type,turn,ground,weather,father,mother,fathers_father,fathers_mother,mothers_father,mothers_mother,rank,sex,age,weigth,weight_change,horse_num,time_diff,first_corner,last_corner
0,202101010101,9,5,1,2018105875,52.0,01181,68.7,34.6,01185,2021-06-12,1,1,1200,芝,右,良,小雨,ダイワメジャー,カシュカシュ,サンデーサイレンス,スカーレットブーケ,アフリート,スリーソウツ,0,牝,3,474,-22,16,0.0,4,4
1,202101010101,6,3,2,2018105258,56.0,01140,68.8,35.0,00438,2021-06-12,1,1,1200,芝,右,良,小雨,エピファネイア,ペルレンケッテ,シンボリクリスエス,シーザリオ,ディープインパクト,プンティラ,0,牡,3,470,-2,16,0.1,1,1
2,202101010101,14,7,3,2018101134,54.0,01096,68.9,34.0,01123,2021-06-12,1,1,1200,芝,右,良,小雨,ミッキーアイル,ザッハトルテ,ディープインパクト,スターアイル,クロフネ,パフェ,0,牝,3,394,2,16,0.2,11,10
3,202101010101,10,5,4,2018102791,54.0,01157,69.0,35.2,01088,2021-06-12,1,1,1200,芝,右,良,小雨,Kingman,ドゥケシーナ,Invincible Spirit,Zenda,Henrythenavigator,Dreamlike,1,牝,3,468,2,16,0.3,2,2
4,202101010101,5,3,5,2018101539,54.0,01015,69.0,34.5,00417,2021-06-12,1,1,1200,芝,右,良,小雨,カレンブラックヒル,ヒシシャトル,ダイワメジャー,チャールストンハーバー,ボストンハーバー,ヒシレイホウ,1,牝,3,456,8,16,0.3,7,7


In [295]:
r.merge_old_results('2019/1/1')
r.data_m

  0%|          | 0/316 [00:00<?, ?it/s]

,race_id,horse_no,frame_no,arriving_order,horse_id,impost,jockey_id,goal_time,last_three_furlong,trainer_id,date,place_id,race_no,distance,race_type,turn,ground,weather,father,mother,fathers_father,fathers_mother,mothers_father,mothers_mother,rank,sex,age,weigth,weight_change,horse_num,time_diff,first_corner,last_corner,p1_place_id,p1_weather,p1_race_no,p1_horse_num,p1_horse_no,p1_arriving_order,p1_race_type,p1_distance,p1_ground,p1_goal_time,p1_time_diff,p1_last_three_furlong,p1_first_corner,p1_last_corner,p2_place_id,p2_weather,p2_race_no,p2_horse_num,p2_horse_no,p2_arriving_order,p2_race_type,p2_distance,p2_ground,p2_goal_time,p2_time_diff,p2_last_three_furlong,p2_first_corner,p2_last_corner
0,202101010101,9,5,1,2018105875,52.0,01181,68.7,34.6,01185,2021-06-12,1,1,1200,芝,右,良,小雨,ダイワメジャー,カシュカシュ,サンデーサイレンス,スカーレットブーケ,アフリート,スリーソウツ,0,牝,3,474,-22,16,0.0,4,4,4.0,晴,5.0,16.0,16.0,6.0,芝,1200.0,良,70.4,0.6,35.2,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,202101010101,6,3,2,2018105258,56.0,01140,68.8,35.0,00438,2021-06-12,1,1,1200,芝,右,良,小雨,エピファネイア,ペルレンケッテ,シンボリクリスエス,シーザリオ,ディープインパクト,プンティラ,0,牡,3,470,-2,16,0.1,1,1,9.0,晴,2.0,15.0,15.0,2.0,ダート,1400.0,良,85.6,0.4,37.8,2.0,2.0,9.0,晴,5.0,16.0,9.0,4.0,芝,1600.0,良,94.4,0.3,34.5,1.0,1.0
2,202101010101,14,7,3,2018101134,54.0,01096,68.9,34.0,01123,2021-06-12,1,1,1200,芝,右,良,小雨,ミッキーアイル,ザッハトルテ,ディープインパクト,スターアイル,クロフネ,パフェ,0,牝,3,394,2,16,0.2,11,10,7.0,雨,3.0,18.0,17.0,10.0,芝,1400.0,不,87.2,1.7,39.3,4.0,6.0,9.0,晴,2.0,12.0,12.0,8.0,芝,1600.0,良,95.3,1.3,35.5,2.0,2.0
3,202101010101,10,5,4,2018102791,54.0,01157,69.0,35.2,01088,2021-06-12,1,1,1200,芝,右,良,小雨,Kingman,ドゥケシーナ,Invincible Spirit,Zenda,Henrythenavigator,Dreamlike,1,牝,3,468,2,16,0.3,2,2,10.0,晴,2.0,18.0,9.0,8.0,芝,1200.0,良,69.2,0.3,35.4,10.0,9.0,10.0,晴,2.0,18.0,18.0,3.0,芝,1200.0,良,68.4,0.2,35.8,2.0,2.0
4,202101010101,5,3,5,2018101539,54.0,01015,69.0,34.5,00417,2021-06-12,1,1,1200,芝,右,良,小雨,カレンブラックヒル,ヒシシャトル,ダイワメジャー,チャールストンハーバー,ボストンハーバー,ヒシレイホウ,1,牝,3,456,8,16,0.3,7,7,4.0,晴,1.0,15.0,3.0,4.0,ダート,1200.0,良,72.7,0.1,37.6,8.0,7.0,4.0,曇,2.0,15.0,13.0,5.0,ダート,1200.0,良,73.2,0.5,38.5,6.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126055,201908040412,5,5,5,2014102848,57.0,01092,111.5,37.1,01079,2019-10-13,8,12,1800,ダート,右,重,晴,ブラックタキシード,ワレンダラッド,サンデーサイレンス,オーピーキャット,ワレンダー,マルゼンラッド,1,牡,5,492,-10,9,0.6,3,3,6.0,曇,10.0,14.0,3.0,5.0,ダート,1800.0,良,113.5,0.6,38.0,9.0,5.0,8.0,晴,9.0,16.0,4.0,1.0,ダート,1800.0,良,111.7,0.0,36.8,3.0,3.0
126056,201908040412,2,2,6,2012100555,57.0,01126,111.8,37.1,01050,2019-10-13,8,12,1800,ダート,右,重,晴,キンシャサノキセキ,ユーキャンドゥイット,フジキセキ,ケルトシャーン,Gold Alert,Royal Fit,1,牡,7,526,2,9,0.9,7,8,9.0,晴,11.0,8.0,6.0,2.0,ダート,1800.0,稍,111.2,1.0,37.1,4.0,3.0,4.0,晴,10.0,15.0,6.0,8.0,ダート,1800.0,良,112.3,1.4,37.2,9.0,12.0
126057,201908040412,1,1,7,2014105543,57.0,01122,111.8,37.4,00438,2019-10-13,8,12,1800,ダート,右,重,晴,クロフネ,スプリングチケット,フレンチデピュティ,ブルーアヴェニュー,トニービン,カズミハルコマ,1,牡,5,498,-4,9,0.9,4,4,6.0,曇,10.0,14.0,4.0,11.0,ダート,1800.0,良,114.5,1.6,38.9,11.0,6.0,8.0,晴,12.0,16.0,4.0,2.0,ダート,1800.0,良,111.7,0.4,38.0,4.0,2.0
126058,201908040412,4,4,8,2013103613,55.0,05429,111.8,37.5,01072,2019-10-13,8,12,1800,ダート,右,重,晴,スペシャルウィーク,サーリセルカ,サンデーサイレンス,キャンペンガール,ウォーエンブレム,ラストリゾート,1,牝,6,504,0,9,0.9,2,1,9.0,晴,10.0,12.0,5.0,4.0,ダート,2000.0,良,124.9,1.1,38.8,2.0,1.0,9.0,晴,11.0,8.0,5.0,3.0,ダート,1800.0,稍,111.5,1.3,37.7,2.0,2.0


In [296]:
r.process_categorical()
r.data_c

,race_id,horse_no,frame_no,arriving_order,horse_id,impost,jockey_id,goal_time,last_three_furlong,trainer_id,date,place_id,race_no,distance,race_type,ground,weather,father,mother,fathers_father,fathers_mother,mothers_father,mothers_mother,rank,age,weigth,weight_change,horse_num,time_diff,first_corner,last_corner,p1_place_id,p1_weather,p1_race_no,p1_horse_num,p1_horse_no,p1_arriving_order,p1_race_type,p1_distance,p1_ground,p1_goal_time,p1_time_diff,p1_last_three_furlong,p1_first_corner,p1_last_corner,p2_place_id,p2_weather,p2_race_no,p2_horse_num,p2_horse_no,p2_arriving_order,p2_race_type,p2_distance,p2_ground,p2_goal_time,p2_time_diff,p2_last_three_furlong,p2_first_corner,p2_last_corner,sex_牝,sex_牡,sex_セ,turn_右,turn_左,turn_他
0,202101010101,9,5,1,16671,52.0,130,68.7,34.6,207,2021-06-12,1,1,1200,1,2,0,396,2065,182,388,626,4098,0,3,474,-22,16,0.0,4,4,4.0,3,5.0,16.0,16.0,6.0,2,1200.0,3,70.4,0.6,35.2,3.0,3.0,NaN,0,NaN,NaN,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,1,0,0,1,0,0
1,202101010101,6,3,2,16221,56.0,91,68.8,35.0,32,2021-06-12,1,1,1200,1,2,0,265,7688,186,383,838,5761,0,3,470,-2,16,0.1,1,1,9.0,3,2.0,15.0,15.0,2.0,1,1400.0,3,85.6,0.4,37.8,2.0,2.0,9.0,3,5.0,16.0,9.0,4.0,2,1600.0,3,94.4,0.3,34.5,1.0,1.0,0,1,0,1,0,0
2,202101010101,14,7,3,13495,54.0,59,68.9,34.0,145,2021-06-12,1,1,1200,1,2,0,510,3505,209,390,697,5126,0,3,394,2,16,0.2,11,10,7.0,5,3.0,18.0,17.0,10.0,2,1400.0,1,87.2,1.7,39.3,4.0,6.0,9.0,3,2.0,12.0,12.0,8.0,2,1600.0,3,95.3,1.3,35.5,2.0,2.0,1,0,0,1,0,0
3,202101010101,10,5,4,14475,54.0,106,69.0,35.2,111,2021-06-12,1,1,1200,1,2,0,100,5719,75,292,235,461,1,3,468,2,16,0.3,2,2,10.0,3,2.0,18.0,9.0,8.0,2,1200.0,3,69.2,0.3,35.4,10.0,9.0,10.0,3,2.0,18.0,18.0,3.0,2,1200.0,3,68.4,0.2,35.8,2.0,2.0,1,0,0,1,0,0
4,202101010101,5,3,5,13703,54.0,22,69.0,34.5,17,2021-06-12,1,1,1200,1,2,0,289,6590,203,412,929,5204,1,3,456,8,16,0.3,7,7,4.0,3,1.0,15.0,3.0,4.0,1,1200.0,3,72.7,0.1,37.6,8.0,7.0,4.0,4,2.0,15.0,13.0,5.0,1,1200.0,3,73.2,0.5,38.5,6.0,6.0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126055,201908040412,5,5,5,1412,57.0,56,111.5,37.1,102,2019-10-13,8,12,1800,0,3,2,484,9943,182,329,994,6124,1,5,492,-10,9,0.6,3,3,6.0,4,10.0,14.0,3.0,5.0,1,1800.0,3,113.5,0.6,38.0,9.0,5.0,8.0,3,9.0,16.0,4.0,1.0,1,1800.0,3,111.7,0.0,36.8,3.0,3.0,0,1,0,1,0,0
126056,201908040412,2,2,6,150,57.0,80,111.8,37.1,74,2019-10-13,8,12,1800,0,3,2,301,8900,219,351,212,1432,1,7,526,2,9,0.9,7,8,9.0,3,11.0,8.0,6.0,2.0,1,1800.0,2,111.2,1.0,37.1,4.0,3.0,4.0,3,10.0,15.0,6.0,8.0,1,1800.0,3,112.3,1.4,37.2,9.0,12.0,0,1,0,1,0,0
126057,201908040412,1,1,7,1853,57.0,76,111.8,37.4,32,2019-10-13,8,12,1800,0,3,2,306,4423,220,457,849,2773,1,5,498,-4,9,0.9,4,4,6.0,4,10.0,14.0,4.0,11.0,1,1800.0,3,114.5,1.6,38.9,11.0,6.0,8.0,3,12.0,16.0,4.0,2.0,1,1800.0,3,111.7,0.4,38.0,4.0,2.0,0,1,0,1,0,0
126058,201908040412,4,4,8,651,55.0,170,111.8,37.5,95,2019-10-13,8,12,1800,0,3,2,376,3496,182,340,649,6565,1,6,504,0,9,0.9,2,1,9.0,3,10.0,12.0,5.0,4.0,1,2000.0,3,124.9,1.1,38.8,2.0,1.0,9.0,3,11.0,8.0,5.0,3.0,1,1800.0,2,111.5,1.3,37.7,2.0,2.0,1,0,0,1,0,0


In [344]:
r.data_c.head()

,race_id,horse_no,frame_no,arriving_order,horse_id,impost,jockey_id,goal_time,last_three_furlong,trainer_id,date,place_id,race_no,distance,race_type,ground,weather,father,mother,fathers_father,fathers_mother,mothers_father,mothers_mother,rank,age,weigth,weight_change,horse_num,time_diff,first_corner,last_corner,p1_place_id,p1_weather,p1_race_no,p1_horse_num,p1_horse_no,p1_arriving_order,p1_race_type,p1_distance,p1_ground,p1_goal_time,p1_time_diff,p1_last_three_furlong,p1_first_corner,p1_last_corner,p2_place_id,p2_weather,p2_race_no,p2_horse_num,p2_horse_no,p2_arriving_order,p2_race_type,p2_distance,p2_ground,p2_goal_time,p2_time_diff,p2_last_three_furlong,p2_first_corner,p2_last_corner,sex_牝,sex_牡,sex_セ,turn_右,turn_左,turn_他
0,202101010101,9,5,1,16671,52.0,130,68.7,34.6,207,2021-06-12,1,1,1200,1,2,0,396,2065,182,388,626,4098,0,3,474,-22,16,0.0,4,4,4.0,3,5.0,16.0,16.0,6.0,2,1200.0,3,70.4,0.6,35.2,3.0,3.0,NaN,0,NaN,NaN,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,1,0,0,1,0,0
1,202101010101,6,3,2,16221,56.0,91,68.8,35.0,32,2021-06-12,1,1,1200,1,2,0,265,7688,186,383,838,5761,0,3,470,-2,16,0.1,1,1,9.0,3,2.0,15.0,15.0,2.0,1,1400.0,3,85.6,0.4,37.8,2.0,2.0,9.0,3,5.0,16.0,9.0,4.0,2,1600.0,3,94.4,0.3,34.5,1.0,1.0,0,1,0,1,0,0
2,202101010101,14,7,3,13495,54.0,59,68.9,34.0,145,2021-06-12,1,1,1200,1,2,0,510,3505,209,390,697,5126,0,3,394,2,16,0.2,11,10,7.0,5,3.0,18.0,17.0,10.0,2,1400.0,1,87.2,1.7,39.3,4.0,6.0,9.0,3,2.0,12.0,12.0,8.0,2,1600.0,3,95.3,1.3,35.5,2.0,2.0,1,0,0,1,0,0
3,202101010101,10,5,4,14475,54.0,106,69.0,35.2,111,2021-06-12,1,1,1200,1,2,0,100,5719,75,292,235,461,1,3,468,2,16,0.3,2,2,10.0,3,2.0,18.0,9.0,8.0,2,1200.0,3,69.2,0.3,35.4,10.0,9.0,10.0,3,2.0,18.0,18.0,3.0,2,1200.0,3,68.4,0.2,35.8,2.0,2.0,1,0,0,1,0,0
4,202101010101,5,3,5,13703,54.0,22,69.0,34.5,17,2021-06-12,1,1,1200,1,2,0,289,6590,203,412,929,5204,1,3,456,8,16,0.3,7,7,4.0,3,1.0,15.0,3.0,4.0,1,1200.0,3,72.7,0.1,37.6,8.0,7.0,4.0,4,2.0,15.0,13.0,5.0,1,1200.0,3,73.2,0.5,38.5,6.0,6.0,1,0,0,1,0,0


In [345]:
df = r.data_c.set_index('race_id')
df = df.dropna()
df = df.drop(['arriving_order', 'goal_time', 'last_three_furlong', 'time_diff', 'first_corner', 'last_corner'], axis=1)
df

,horse_no,frame_no,horse_id,impost,jockey_id,trainer_id,date,place_id,race_no,distance,race_type,ground,weather,father,mother,fathers_father,fathers_mother,mothers_father,mothers_mother,rank,age,weigth,weight_change,horse_num,p1_place_id,p1_weather,p1_race_no,p1_horse_num,p1_horse_no,p1_arriving_order,p1_race_type,p1_distance,p1_ground,p1_goal_time,p1_time_diff,p1_last_three_furlong,p1_first_corner,p1_last_corner,p2_place_id,p2_weather,p2_race_no,p2_horse_num,p2_horse_no,p2_arriving_order,p2_race_type,p2_distance,p2_ground,p2_goal_time,p2_time_diff,p2_last_three_furlong,p2_first_corner,p2_last_corner,sex_牝,sex_牡,sex_セ,turn_右,turn_左,turn_他
race_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
202101010101,6,3,16221,56.0,91,32,2021-06-12,1,1,1200,1,2,0,265,7688,186,383,838,5761,0,3,470,-2,16,9.0,3,2.0,15.0,15.0,2.0,1,1400.0,3,85.6,0.4,37.8,2.0,2.0,9.0,3,5.0,16.0,9.0,4.0,2,1600.0,3,94.4,0.3,34.5,1.0,1.0,0,1,0,1,0,0
202101010101,14,7,13495,54.0,59,145,2021-06-12,1,1,1200,1,2,0,510,3505,209,390,697,5126,0,3,394,2,16,7.0,5,3.0,18.0,17.0,10.0,2,1400.0,1,87.2,1.7,39.3,4.0,6.0,9.0,3,2.0,12.0,12.0,8.0,2,1600.0,3,95.3,1.3,35.5,2.0,2.0,1,0,0,1,0,0
202101010101,10,5,14475,54.0,106,111,2021-06-12,1,1,1200,1,2,0,100,5719,75,292,235,461,1,3,468,2,16,10.0,3,2.0,18.0,9.0,8.0,2,1200.0,3,69.2,0.3,35.4,10.0,9.0,10.0,3,2.0,18.0,18.0,3.0,2,1200.0,3,68.4,0.2,35.8,2.0,2.0,1,0,0,1,0,0
202101010101,5,3,13703,54.0,22,17,2021-06-12,1,1,1200,1,2,0,289,6590,203,412,929,5204,1,3,456,8,16,4.0,3,1.0,15.0,3.0,4.0,1,1200.0,3,72.7,0.1,37.6,8.0,7.0,4.0,4,2.0,15.0,13.0,5.0,1,1200.0,3,73.2,0.5,38.5,6.0,6.0,1,0,0,1,0,0
202101010101,2,1,13142,54.0,6,64,2021-06-12,1,1,1200,1,2,0,259,3408,209,337,697,3485,1,3,456,4,16,4.0,4,5.0,15.0,12.0,11.0,1,1200.0,4,73.7,1.5,38.9,2.0,2.0,4.0,3,1.0,15.0,13.0,3.0,1,1200.0,3,73.5,0.6,38.6,2.0,1.0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201908040412,5,5,1412,57.0,56,102,2019-10-13,8,12,1800,0,3,2,484,9943,182,329,994,6124,1,5,492,-10,9,6.0,4,10.0,14.0,3.0,5.0,1,1800.0,3,113.5,0.6,38.0,9.0,5.0,8.0,3,9.0,16.0,4.0,1.0,1,1800.0,3,111.7,0.0,36.8,3.0,3.0,0,1,0,1,0,0
201908040412,2,2,150,57.0,80,74,2019-10-13,8,12,1800,0,3,2,301,8900,219,351,212,1432,1,7,526,2,9,9.0,3,11.0,8.0,6.0,2.0,1,1800.0,2,111.2,1.0,37.1,4.0,3.0,4.0,3,10.0,15.0,6.0,8.0,1,1800.0,3,112.3,1.4,37.2,9.0,12.0,0,1,0,1,0,0
201908040412,1,1,1853,57.0,76,32,2019-10-13,8,12,1800,0,3,2,306,4423,220,457,849,2773,1,5,498,-4,9,6.0,4,10.0,14.0,4.0,11.0,1,1800.0,3,114.5,1.6,38.9,11.0,6.0,8.0,3,12.0,16.0,4.0,2.0,1,1800.0,3,111.7,0.4,38.0,4.0,2.0,0,1,0,1,0,0


In [306]:
from data_processor import split_data

In [346]:
train, test = split_data(df)
train, valid = split_data(train)

In [347]:
X_train = train.drop(['rank', 'date'], axis=1)
y_train = train['rank']
X_valid = valid.drop(['rank', 'date'], axis=1)
y_valid = valid['rank']
X_test = test.drop(['rank', 'date'], axis=1)
y_test = test['rank']

In [ ]:
import optuna.integration.lightgbm as lgb_o

lgb_train = lgb_o.Dataset(X_train, y_train)
lgb_valid = lgb_o.Dataset(X_valid, y_valid)

params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 3,
    'metric': 'multi_logloss'
}

lgb_clf_o = lgb_o.train(params, lgb_train, valid_sets=[lgb_train, lgb_valid], verbose_eval=100, early_stopping_rounds=10)

In [305]:
print(accuracy_score(y_test, y_pred))

0.764406833180713


In [349]:
lgb_clf_o.params

{'boosting_type': 'gbdt',
 'objective': 'multiclass',
 'num_class': 3,
 'metric': 'multi_logloss',
 'feature_pre_filter': False,
 'lambda_l1': 3.3996858124765543e-06,
 'lambda_l2': 1.9986700967376345e-05,
 'num_leaves': 100,
 'feature_fraction': 0.4,
 'bagging_fraction': 0.4954480747855715,
 'bagging_freq': 3,
 'min_child_samples': 20,
 'num_iterations': 1000,
 'early_stopping_round': 10,
 'categorical_column': [9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  23,
  28,
  30,
  37,
  42,
  44]}

In [337]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 3,
    'metric': 'multi_logloss',
    'feature_pre_filter': False,
    'lambda_l1': 3.3996858124765543e-06,
    'lambda_l2': 1.9986700967376345e-05,
    'num_leaves': 100,
    'feature_fraction': 0.4,
    'bagging_fraction': 0.4954480747855715,
    'bagging_freq': 3,
    'min_child_samples': 20,
    'num_iterations': 1000,
    'categorical_column': [9,
    10,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
    23,
    28,
    30,
    37,
    42,
    44]
}

In [350]:
import lightgbm as lgb
model_o = lgb.LGBMClassifier(**params)
model_o.fit(X_train, y_train)

LGBMClassifier(metric='multi_logloss', num_class=3, objective='multiclass')

In [351]:
y_pred_o = model_o.predict(X_test)
y_pred_prob_o = model_o.predict_proba(X_test)

In [352]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred_o))

0.49227951624617133


In [359]:
df_pred_prob_o = pd.DataFrame({'y':y_test, 'target0_prob':y_pred_prob_o[:,0], 'target1_prob':y_pred_prob_o[:,1], 'target2_prob':y_pred_prob_o[:,2]})
df_pred_prob_o

,y,target0_prob,target1_prob,target2_prob
race_id,,,,
202106010708,0,0.539506,0.234070,0.226424
202106010708,0,0.399229,0.410821,0.189950
202106010708,0,0.213382,0.500964,0.285654
202106010708,1,0.261432,0.563546,0.175022
202106010708,1,0.250920,0.426472,0.322609
...,...,...,...,...
202105050809,2,0.273449,0.232947,0.493605
202105050809,2,0.082205,0.566803,0.350992
202105050809,2,0.069810,0.431543,0.498647


In [364]:
df_pred_prob_o.head(20)

,y,target0_prob,target1_prob,target2_prob
race_id,,,,
202106010708,0,0.539506,0.234070,0.226424
202106010708,0,0.399229,0.410821,0.189950
202106010708,0,0.213382,0.500964,0.285654
202106010708,1,0.261432,0.563546,0.175022
202106010708,1,0.250920,0.426472,0.322609
202106010708,1,0.126728,0.380066,0.493206
202106010708,1,0.302250,0.280395,0.417355
202106010708,1,0.069232,0.315367,0.615401
202106010708,2,0.075598,0.554984,0.369418


In [354]:
importances = pd.DataFrame({'features':X_train.columns, 'importance':model_o.feature_importances_})
importances.sort_values('importance', ascending=False)[:20]

,features,importance
17,mothers_mother,2231
16,mothers_father,2009
13,mother,1263
12,father,1135
15,fathers_mother,429
21,horse_num,302
27,p1_arriving_order,269
41,p2_arriving_order,191
32,p1_time_diff,133
7,race_no,114
